In [1]:
import pandas as pd
import nltk
import re

In [3]:
data = pd.read_csv("/content/drive/MyDrive/data/spam.csv",encoding = "ISO-8859-1")

In [4]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1,inplace = True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
nltk.download('stopwords')
nltk.download('all')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]   

True

In [7]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [8]:
ps = PorterStemmer()
input = []

In [9]:
data.shape

(5572, 2)

In [10]:
from nltk.translate.ribes_score import word_rank_alignment
from numpy.lib.shape_base import split

for i in range(0,5572):
  v2 = data['v2'][i]

  #removing punctuation
  v2 = re.sub('[^a-zA-Z]',' ',v2)

  #converting to lower case
  v2 = v2.lower()

  #splitting the sentence
  v2 = v2.split()

  #removing the stopwords and stemming
  v2 = [ps.stem(word) for word in v2 if not word in set(stopwords.words('english'))]

  v2 = ' '.join(v2)

  input.append(v2)

In [11]:
#creating document term matrix

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000)

In [12]:
cv

CountVectorizer(max_features=2000)

In [13]:
x = cv.fit_transform(input).toarray()


In [14]:
x
x.shape

(5572, 2000)

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

data['v1'] = le.fit_transform(data['v1'])
data['v1'].unique()

array([0, 1])

In [16]:
y = data['v1'].values

In [17]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [18]:
y = y.reshape(-1,1)

In [19]:
#y = data.iloc[:,1:2].values
#y

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4)

In [22]:
from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Activation,Flatten
from keras.models import Model
from tensorflow.keras.models import Sequential
model = Sequential()

In [23]:
model.add(Dense(1565,activation = "relu"))
model.add(Dense(3000,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.add(Flatten())

In [24]:
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(x_train,y_train,epochs = 15)

Epoch 1/15
105/105 [==============================] - 5s 35ms/step - loss: 0.1288 - accuracy: 0.9581
Epoch 2/15
105/105 [==============================] - 4s 39ms/step - loss: 0.0157 - accuracy: 0.9973
Epoch 3/15
105/105 [==============================] - 4s 38ms/step - loss: 0.0027 - accuracy: 0.9997
Epoch 4/15
105/105 [==============================] - 5s 48ms/step - loss: 0.0021 - accuracy: 0.9997
Epoch 5/15
105/105 [==============================] - 4s 34ms/step - loss: 0.0020 - accuracy: 0.9997
Epoch 6/15
105/105 [==============================] - 4s 34ms/step - loss: 0.0019 - accuracy: 0.9997
Epoch 7/15
105/105 [==============================] - 4s 35ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 8/15
105/105 [==============================] - 4s 34ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 9/15
105/105 [==============================] - 4s 34ms/step - loss: 0.0017 - accuracy: 0.9997
Epoch 10/15
105/105 [==============================] - 5s 46ms/step - loss: 0.0015 - accura

In [25]:
model.save("SMS_Spam_Classifier.h5")
ham = "im donee. come pick me up"
spam = "WINNER$$$$ SMS REPLY 'WIN'"
message = re.sub('[^a-zA-Z]',' ',spam)
message

'WINNER     SMS REPLY  WIN '

In [26]:
message = message.split()
message = [ps.stem(word) for word in message if not word in set(stopwords.words('english')) ]
message = ' '.join(message)
message

'winner sm repli win'

In [27]:
cv.transform([message])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [28]:
message1 = cv.transform([message])
message1

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [29]:
import numpy as np
pred = model.predict(message1.astype(float))

1/1 [==============================] - 0s 106ms/step


In [30]:
pred

array([[0.9993413]], dtype=float32)

In [31]:
pred > 0.5

array([[ True]])

In [32]:
msg = re.sub('[^a-zA-Z]',' ',ham)
msg

'im donee  come pick me up'

In [34]:
msg = msg.split()
msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
msg = ' '.join(msg)
msg

'im done come pick'

In [35]:
cv.transform([msg])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [36]:
pred1 = model.predict(cv.transform([msg]))

1/1 [==============================] - 0s 58ms/step


In [37]:
pred1

array([[6.2694113e-07]], dtype=float32)

In [38]:
pred1 > 0.5

array([[False]])